In [ ]:
import torch
import torchvision.transforms as transforms
import torchvision.datasets as datasets
import numpy as np
from PIL import Image
import matplotlib.pyplot as plt
from torch.utils.data import DataLoader
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
import seaborn as sns
import os
from tqdm import tqdm
import cv2

SEED = 123
def set_seed(seed):
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False

set_seed(SEED)


In [ ]:

INPUT_SIZE = (227, 227)
MEAN = (0.5960, 0.4489, 0.4046)
STD = (0.2102, 0.1782, 0.1719)

main_data_dir = r""
val_dir = os.path.join(main_data_dir, "val")
test_dir = os.path.join(main_data_dir, "test")

val_dataset = datasets.ImageFolder(root=val_dir)
class_names = list(val_dataset.classes)
print("Class to label mapping:", val_dataset.class_to_idx)


In [ ]:
class CLAHETransform:
    def __init__(self, clip_limit=2.0, tile_grid_size=(8, 8)):
        self.clip_limit = clip_limit
        self.tile_grid_size = tile_grid_size

    def __call__(self, img):
        img_np = np.array(img)

        img_lab = cv2.cvtColor(img_np, cv2.COLOR_RGB2LAB)
        l, a, b = cv2.split(img_lab)

        clahe = cv2.createCLAHE(clipLimit=self.clip_limit, tileGridSize=self.tile_grid_size)
        l_clahe = clahe.apply(l)

        img_clahe = cv2.merge((l_clahe, a, b))
        img_clahe = cv2.cvtColor(img_clahe, cv2.COLOR_LAB2RGB)

        return Image.fromarray(img_clahe)

In [ ]:
transform_val_test = transforms.Compose([
    transforms.Resize(INPUT_SIZE),
    CLAHETransform(clip_limit=2.0, tile_grid_size=(8, 8)),
    transforms.ToTensor(),
    transforms.Normalize(mean=MEAN, std=STD)
])

val_dataset = datasets.ImageFolder(root=val_dir, transform=transform_val_test)
test_dataset = datasets.ImageFolder(root=test_dir, transform=transform_val_test)

val_loader = DataLoader(val_dataset, shuffle=False)
test_loader = DataLoader(test_dataset,  shuffle=False)

In [ ]:
import torch.nn as nn
import torchvision.models as models


class AlexNet(nn.Module):
    def __init__(self, num_classes=len(class_names)):
        super(AlexNet, self).__init__()
        self.model = models.alexnet(weights=None) 
        self.model.classifier[6] = nn.Linear(4096, num_classes)  

    def forward(self, x):
        return self.model(x)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = AlexNet(num_classes=len(class_names)).to(device)

model_path = r""
model.load_state_dict(torch.load(model_path, weights_only=True))
model.eval() 

print(f"Loaded model from {model_path}")
print(model)

# from torchsummary import summary
# input_size = (3, 227, 227)
# summary(model, input_size=input_size, device=str(device))

In [ ]:
import torch
import torch.nn.functional as F
from tqdm import tqdm
import matplotlib.pyplot as plt
import seaborn as sns


def evaluate_model(model, data_loader, device, class_names, set_name, export_txt=False):
    model.eval()
    y_true = []
    y_pred = []
    image_paths = []  # To store image file paths
    confidence_scores = []  # To store confidence percentages

    dataset = data_loader.dataset

    with torch.no_grad():
        for idx, (inputs, labels) in enumerate(tqdm(data_loader, desc=f"Evaluating {set_name} Set")):
            inputs, labels = inputs.to(device), labels.to(device)
            outputs = model(inputs)  # Get raw logits
            
            # Convert logits to probabilities using softmax
            probabilities = torch.softmax(outputs, dim=1)
            
            # Get the predicted class and the associated confidence score
            max_probs, predicted = torch.max(probabilities, 1)
            y_true.extend(labels.cpu().numpy())
            y_pred.extend(predicted.cpu().numpy())
            confidence_scores.extend(max_probs.cpu().numpy() * 100)  # Convert to percentage

            # Collect image file paths from the dataset
            image_paths.extend([dataset.samples[i][0] for i in range(idx * data_loader.batch_size, (idx + 1) * data_loader.batch_size)])

    # Calculate accuracy
    accuracy = accuracy_score(y_true, y_pred)
    print(f"\n{set_name} Accuracy: {accuracy * 100:.2f}%")

    print(f"\n{set_name} Classification Report:\n")
    print(classification_report(y_true, y_pred, target_names=class_names))

    # Generate the confusion matrix
    cm = confusion_matrix(y_true, y_pred)
    plt.figure(figsize=(10, 8))
    sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', xticklabels=class_names, yticklabels=class_names)
    plt.title(f'{set_name} Confusion Matrix on AlexNet')
    plt.xlabel('Predicted Labels')
    plt.ylabel('True Labels')
    plt.show()

    # Export predictions to a text file if export_txt is True
    if export_txt:
        output_file = f"{set_name}_alexnet_predictions_with_confidence.txt"
        with open(output_file, "w") as file:
            file.write("Image File, True Label, Predicted Label, Confidence (%)\n")
            for img_path, true_label, pred_label, confidence in zip(image_paths, y_true, y_pred, confidence_scores):
                true_class_name = class_names[true_label]
                pred_class_name = class_names[pred_label]
                file.write(f"{img_path}, {true_class_name}, {pred_class_name}, {confidence:.2f}%\n")
        print(f"Predictions saved to {output_file}")


In [ ]:
evaluate_model(model, val_loader, device, class_names, "Validation")

In [ ]:
evaluate_model(model, test_loader, device, class_names, "Test", export_txt=True)